# Azure埋め込みの例

> 注意: openaiライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 をご覧ください

この例では、Azure OpenAIサービスを使用した埋め込みについて説明します。

## セットアップ

まず、必要な依存関係をインストールします。

```bash
! pip install "openai>=0.28.1,<1.0.0"
```

以下のセクションが正しく動作するために、まず最初にいくつかの設定を行う必要があります。`api_base`と`api_version`から始めましょう。`api_base`を見つけるには、https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」→「キーとエンドポイント」の下にある「エンドポイント」の値を確認してください。

In [ ]:
import os
import openai

In [ ]:
openai.api_version = '2023-05-15'
openai.api_base = '' # Please add your endpoint here

次に、`api_type`と`api_key`を設定する必要があります。キーはポータルから取得するか、Microsoft Active Directory認証を通じて取得することができます。これに応じて、`api_type`は`azure`または`azure_ad`のいずれかになります。

### セットアップ: ポータル
まず、ポータルからキーを取得する方法を見てみましょう。https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」→「キーとエンドポイント」の下で「キー」の値の一つを探してください。

In [ ]:
openai.api_type = 'azure'
openai.api_key = os.environ["OPENAI_API_KEY"]

> 注意: この例では、コード内で変数を設定することでライブラリがAzure APIを使用するように設定しました。開発時には、代わりに環境変数を設定することを検討してください：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

### （オプション）セットアップ：Microsoft Active Directory認証
Microsoft Active Directory認証を使用してキーを取得する方法を見てみましょう。ポータルからのキーの代わりにActive Directory認証を使用したい場合は、以下のコードのコメントアウトを解除してください。

In [ ]:
# from azure.identity import DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token

トークンは一定期間有効で、その後期限切れになります。すべてのリクエストで有効なトークンが送信されるようにするには、`requests.auth`にフックして期限切れのトークンを更新することができます：

In [ ]:
import typing
import time
import requests
if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

session = requests.Session()
session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

openai.requestssession = session

## デプロイメント
このセクションでは、埋め込みを作成するために使用できるデプロイメントを作成します。

### デプロイメント: 手動で作成する
`text-similarity-curie-001`モデルを使用してデプロイメントを作成しましょう。ポータルでリソースの「リソース管理」→「モデルデプロイメント」に移動して、新しいデプロイメントを作成します。

In [ ]:
deployment_id = '' # Fill in the deployment id from the portal here

### デプロイメント: 一覧表示
新しいデプロイメントの作成には時間がかかるため、サブスクリプション内で既に完了し、成功したデプロイメントを確認してみましょう。

In [ ]:
print('While deployment running, selecting a completed one that supports embeddings.')
deployment_id = None
result = openai.Deployment.list()
for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

if not deployment_id:
    print('No deployment with status: succeeded found.')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

### Embeddings
それでは、サンプルのembeddingをデプロイメントに送信してみましょう。

In [ ]:
embeddings = openai.Embedding.create(deployment_id=deployment_id,
                                     input="The food was delicious and the waiter...")
                                
print(embeddings)